# Probabilistic logic gates

This Python Jupyter notebook explores noisy logic gates, such as AND and OR. 

The equations came from Diez, F. J.; Druzdzel, M. J.; __"Canonical Probabilistic Models for Knowledge Engineering"__, Technical Report CISIAD-06-01, April 28, 2007.

## Introduction

A discrete Bayesian network with one child node and $n$ parent nodes requires the child node to have a CPT with $2^n$ parameters. Therefore, the number of parameters is exponential in the number of parents. The rapid increase in the number of parameters can be daunting for an expert. If the parameters are learnt from data, there may be few or no examples of certain configurations of the parents. Equally, the expert may never have seen certain configurations. 

To reduce the complexity of the elicitation of numerical probabilities, *canonical models* can be employed.

In practical applications of a Bayesian network, it may not be feasible or desirable to model all of the parent nodes influencing a child node. There may be missing parent nodes due to unknown causal interactions, a lack of data or knowledge. This extra uncertainty in those missing parent nodes can be modelled through the use of *leaky* models.

The output variable $y$ takes on a positive (true) or negative (false) value, denoted $+y$ and $\neg y$ respectively.

The product of an empty set $\prod_{ \emptyset } (\cdot) $ is defined to be 1.

The notation $i \in I_{+}(x)$ is the set of $i$ where $x_i = +x_i$. Similarly, $i \in I_{\neg}(x)$ is the set of $i$ where $x_i = \neg x_i$.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def add_input(p=None):
    """Add another input to a defined binary table."""
    
    if p == None:
        return [[0], [1]]
    else:
        
        r = []
        for start in [0, 1]:
            for row in p:
                row_copy = row[:]
                row_copy.insert(0, start)
                r.append(row_copy)
            
        return r
    
# Tests
assert add_input() == [[0], [1]]
assert add_input([[0], [1]]) == [[0, 0], [0, 1], [1, 0], [1, 1]]
assert add_input([[0, 0], [0, 1], [1, 0], [1, 1]]) == [[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

In [3]:
def generate_combinations(n):
    """Generate all of the possible binary inputs for a function with n inputs."""
    
    assert isinstance(n, int)
    assert n > 0
    
    r = add_input()
    
    for i in range(1,n):
        r = add_input(r)
    
    return r

# Tests
assert generate_combinations(1) == [[0], [1]]
assert generate_combinations(2) == [[0, 0], [0, 1], [1, 0], [1, 1]]
assert generate_combinations(3) == [[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

In [4]:
def build_cpt(n, fn):
    """Build the CPT for a function with n inputs."""
    
    assert isinstance(n, int)
    assert n >= 0
    
    input_combinations = generate_combinations(n)
    
    m = []
    
    for x in input_combinations:
        p = fn(x)
        x.append(p)
        m.append(x)
        
    df = pd.DataFrame(m)
        
    # Rename the input columns
    for i in range(n):
        df = df.rename(columns={i: f'x{i}'})
    
    # Rename the probability column
    df = df.rename(columns={n: 'p(y|x)'})
    
    return df    

## Deterministic NOT gate

The CPT for a perfect NOT gate is given by

$$
p(+y|x) = \left\{
\begin{array}{ll}
      0 & x = +x \\
      1 & x = \neg x \\
\end{array} 
\right.
$$

In [5]:
def perfect_not(x):
    assert x == [0] or x == [1], f"Got {x}"
    
    return 1 if x == [0] else 0

In [6]:
build_cpt(1, perfect_not)

,x0,p(y|x)
0,0,1
1,1,0


## Deterministic OR

In [7]:
def perfect_or(x):
    
    # Preconditions
    assert all([xi == 0 or xi == 1 for xi in x])
    
    return 1 if np.sum(x) > 0 else 0    

In [8]:
build_cpt(2, perfect_or)

,x0,x1,p(y|x)
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,1


In [9]:
build_cpt(3, perfect_or)

,x0,x1,x2,p(y|x)
0,0,0,0,0
1,0,0,1,1
2,0,1,0,1
3,0,1,1,1
4,1,0,0,1
5,1,0,1,1
6,1,1,0,1
7,1,1,1,1


## Noisy OR

Each $X_i$ represents a cause of $Y$. To handle the case where a cause fails to produce an effect when present, a *noisy* version is used.

The CPT for the Noisy OR is defined using:

$$
p(\neg y|x) = \prod_{i \in I_{+}(x)} (1 - c_i)
$$

$$
p(+y|x) = 1 - \prod_{i \in I_{+}(x)} (1 - c_i)
$$

The probability $c_i$ is the probability of $+y$ given that all of the other inputs are false.

In [10]:
def noisy_or(c, x):
    
    # Preconditions
    assert len(c) == len(x)
    assert all([0 <= ci <= 1 for ci in c])
    assert all([xi == 0 or xi == 1 for xi in x])
    
    prod = 1
    for i in range(len(c)):
        if x[i] == 1:
            prod *= (1 - c[i])
    
    return 1 - prod

In [11]:
def build_noisy_or_fn(c):
    def fn(x):
        return noisy_or(c, x)
    return fn

In [21]:
c = [0.1, 0.1, 0.1, 0.1, 0.1]
build_cpt(5, build_noisy_or_fn(c))

,x0,x1,x2,x3,x4,p(y|x)
0,0,0,0,0,0,0.00000
1,0,0,0,0,1,0.10000
2,0,0,0,1,0,0.10000
3,0,0,0,1,1,0.19000
4,0,0,1,0,0,0.10000
5,0,0,1,0,1,0.19000
6,0,0,1,1,0,0.19000
7,0,0,1,1,1,0.27100
8,0,1,0,0,0,0.10000
9,0,1,0,0,1,0.19000


If $c_i = 1 \, \forall i$ then the noisy OR has the same CPT as the deterministic OR as shown below.

In [13]:
c = [1.0, 1.0]
build_cpt(2, build_noisy_or_fn(c))

,x0,x1,p(y|x)
0,0,0,0.0
1,0,1,1.0
2,1,0,1.0
3,1,1,1.0


## Leaky OR

The probability $c_i$ is the probability of $+y$ given that all of the other inputs are false.

$c_L$ (leak probability) is the probability of $+y$ when all explicit causes are absent.

$$
p(\neg y|x) = (1 - c_{L}) \prod_{i \in I_{+}(x)} (1 - c_i)
$$

$$
p(+y|x) = 1 - (1 - c_{L}) \prod_{i \in I_{+}(x)} (1 - c_i)
$$

In [14]:
def leaky_or(c, c_L, x):
    
    # Preconditions
    assert len(c) == len(x)
    assert all([0 <= ci <= 1 for ci in c])
    assert 0 <= c_L <= 1
    assert all([xi == 0 or xi == 1 for xi in x])
    
    prod = 1
    for i in range(len(c)):
        if x[i] == 1:
            prod *= (1 - c[i])
    
    return 1 - (1 - c_L) * prod

In [15]:
def build_leaky_or_fn(c, c_L):
    def fn(x):
        return leaky_or(c, c_L, x)
    return fn

In [16]:
c = [0.8, 0.7]
c_L = 0.2
build_cpt(2, build_leaky_or_fn(c, c_L))

,x0,x1,p(y|x)
0,0,0,0.200
1,0,1,0.760
2,1,0,0.840
3,1,1,0.952


When $c_L = 0$, the CPT is equivalent to the noisy OR. As the noisy OR in turn can represent the deterministic OR, the leaky OR is the most general OR model.

If $c_L = 1$, then $p(y|x) = 1$ for all cases of $x$, as shown below.

In [17]:
build_cpt(2, build_leaky_or_fn(c, 1))

,x0,x1,p(y|x)
0,0,0,1.0
1,0,1,1.0
2,1,0,1.0
3,1,1,1.0


## Simple AND

$$
p(+y|x) = \left\{
\begin{array}{ll}
      c & x = (+x_0, +x_1, \cdots, +x_{N-1}) \\
      s & \textrm{otherwise} \\
\end{array} 
\right.
$$

In [18]:
def simple_and(c, s, x):
    
    assert 0 <= c <= 1
    assert 0 <= s <= 1
    assert all([xi == 0 or xi == 1 for xi in x])
    
    for i in range(len(x)):
        if x[i] != 1:
            return s
    
    return c

In [19]:
def build_simple_and(c, s):
    def fn(x):
        return simple_and(c, s, x)
    return fn

In [20]:
c = 0.8
s = 0.1
build_cpt(2, build_simple_and(c, s))

,x0,x1,p(y|x)
0,0,0,0.1
1,0,1,0.1
2,1,0,0.1
3,1,1,0.8


## Threshold function

The *threshold function* returns true when at least $r$ of its $n$ arguments are true. In the extreme cases the functions OR ($r = 1$) and AND ($r = n$) are handled.

## Noisy MAX